#We use a BERT model pretrained for sentiment analysis of text
[distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english?text=I+like+you.+I+love+you)

We use this model to extract features and train them on our data

In [ ]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/meme_classification_data/train.csv')
df_train.head(3)

,ID,image id,text,label,label_num
0,1,image_2455.jpg,- It is not our fight - Are we not part of thi...,troll,2
1,2,image_3701.jpg,THAT'S THE DIFFERENCE BETWEEN YOU AND ME YOU...,none,0
2,3,image_4166.png,- WHAT DO THE TITANIC AND THE SIXTH SENSE HAVE...,none,0


#Using the model's scores for sentiment classification

##Loading the model

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
MODEL = 'distilbert-base-uncased-finetuned-sst-2-english'

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


##Extracting Sentiment Scores

In [ ]:
X = []
for idx, row in df_train.iterrows():
  text = row['text']
  encoded_input = tokenizer(text, return_tensors='tf')
  output = model(encoded_input)
  scores = output[0][0].numpy()
  X.append(scores)
X=np.array(X)


##Labelling the data


In [ ]:
#We label the troll samples as negative sentiment, and positive sentiment otherwise
def sent_label(label):
  if label == 2:
    return 0
  else:
    return 1
df_train['Sentiment'] = df_train['label_num'].apply(sent_label)
df_train.tail(5)

,ID,image id,text,label,label_num,Sentiment
1986,1987,image_2703.png,Yup still fabulous THEFRESHPICKLE,none,0,1
1987,1988,image_5241.png,ZILLOW SAID YOUR HOME WAS WORTH WHAT?!,troll,2,0
1988,1989,image_4410.jpg,ZUCKERBERG GETS MONEY AND THE ANNOYING PEOPLE ...,hilarious,1,1
1989,1990,image_4996.jpg,#NAME?,hilarious,1,1
1990,1991,image_5609.png,#NAME?,hilarious,1,1


In [ ]:
y = df_train['Sentiment']
one_count = np.count_nonzero(y)
zero_count = y.shape[0]-one_count
print("Class 0:",zero_count,"Samples")
print("Class 1:",one_count,"Samples")

####  Imbalanced Dataset  ####

Class 0: 698 Samples
Class 1: 1293 Samples


##Tree Based Algorithms for classifcation
Since we have only 2 features and an imbalanced dataset we use tree based classifiers and gradient boost classifiers

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
models=[RandomForestClassifier(),DecisionTreeClassifier(), GradientBoostingClassifier(), XGBClassifier()]
names = ['Random-Forest','Decision-Tree','GradientBoosting', 'XGB']
for i in range(len(models)):
  models[i].fit(X_train,y_train)
  print("MODEL: ", names[i])
  print(classification_report(y_test, models[i].predict(X_test)))

MODEL:  Random-Forest
              precision    recall  f1-score   support

           0       0.50      0.06      0.11       146
           1       0.64      0.96      0.77       253

    accuracy                           0.63       399
   macro avg       0.57      0.51      0.44       399
weighted avg       0.59      0.63      0.53       399

MODEL:  Decision-Tree
              precision    recall  f1-score   support

           0       0.35      0.33      0.34       146
           1       0.63      0.65      0.64       253

    accuracy                           0.53       399
   macro avg       0.49      0.49      0.49       399
weighted avg       0.53      0.53      0.53       399

MODEL:  GradientBoosting
              precision    recall  f1-score   support

           0       0.40      0.13      0.20       146
           1       0.64      0.89      0.74       253

    accuracy                           0.61       399
   macro avg       0.52      0.51      0.47       399
weigh

#Extracting Features from the model 

In [ ]:
!pip install sentence-transformers

In [ ]:
import nltk
from sentence_transformers import SentenceTransformer
st = SentenceTransformer('distilbert-base-uncased-finetuned-sst-2-english') 
sentiment_feats_train = []
for index, row in df_train.iterrows():
  sentiment_feats_train.append(st.encode(row['text']))

Exception when trying to download http://sbert.net/models/distilbert-base-uncased-finetuned-sst-2-english.zip. Response 404
SentenceTransformer-Model http://sbert.net/models/distilbert-base-uncased-finetuned-sst-2-english.zip not found. Try to create it from scratch
Try to create Transformer Model distilbert-base-uncased-finetuned-sst-2-english with mean pooling


In [ ]:
import numpy as np
sentiment_feats_train = np.array(sentiment_feats_train)
sentiment_feats_train.shape

(1991, 768)

##Saving the features

In [ ]:
sentiment_train = pd.DataFrame(sentiment_feats_train, columns = range(768))
sentiment_train.to_csv('/content/drive/MyDrive/meme_classification_data/train_Sentiment_feats.csv', index=False)

#Training the features on NN

In [ ]:
from sklearn.model_selection import train_test_split
X = sentiment_feats_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop,Adam
model=Sequential() 
model.add(Dense(10,activation='sigmoid',input_shape=(768,),kernel_initializer='he_normal'))
#model.add(Dense(20,activation='sigmoid'))
model.add(Dense(5,activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 10)                7690      
_________________________________________________________________
dense_24 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 6         
Total params: 7,751
Trainable params: 7,751
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy','AUC'])


history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=40,
                    verbose=1,
                    validation_data=(X_val, y_val))


Epoch 1/40
44/44 [==============================] - 1s 11ms/step - loss: 0.7919 - accuracy: 0.3781 - auc: 0.5257 - val_loss: 0.7170 - val_accuracy: 0.3015 - val_auc: 0.4612
Epoch 2/40
44/44 [==============================] - 0s 5ms/step - loss: 0.7003 - accuracy: 0.4069 - auc: 0.5309 - val_loss: 0.6759 - val_accuracy: 0.6985 - val_auc: 0.4790
Epoch 3/40
44/44 [==============================] - 0s 5ms/step - loss: 0.6764 - accuracy: 0.6445 - auc: 0.5024 - val_loss: 0.6526 - val_accuracy: 0.6985 - val_auc: 0.4894
Epoch 4/40
44/44 [==============================] - 0s 5ms/step - loss: 0.6648 - accuracy: 0.6360 - auc: 0.5254 - val_loss: 0.6376 - val_accuracy: 0.6985 - val_auc: 0.4996
Epoch 5/40
44/44 [==============================] - 0s 5ms/step - loss: 0.6565 - accuracy: 0.6412 - auc: 0.5212 - val_loss: 0.6276 - val_accuracy: 0.6985 - val_auc: 0.5171
Epoch 6/40
44/44 [==============================] - 0s 5ms/step - loss: 0.6525 - accuracy: 0.6425 - auc: 0.5599 - val_loss: 0.6244 - val_ac

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test, batch_size=64, verbose=1)
p = y_pred.mean()
y_pred = np.where(y_pred >= p , 1 , 0)
print(y.shape, np.count_nonzero(y_pred))
print(classification_report(y_test,y_pred))

7/7 [==============================] - 0s 2ms/step
(1991,) 246
              precision    recall  f1-score   support

           0       0.44      0.46      0.45       146
           1       0.68      0.66      0.67       253

    accuracy                           0.59       399
   macro avg       0.56      0.56      0.56       399
weighted avg       0.59      0.59      0.59       399



#Training on Tree based classifiers and gradient boost classifiers

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
models=[RandomForestClassifier(),DecisionTreeClassifier(), GradientBoostingClassifier(), XGBClassifier()]
names = ['Random-Forest','Decision-Tree','GradientBoosting', 'XGB']
for i in range(len(models)):
  models[i].fit(X_train,y_train)
  print("MODEL: ", names[i])
  print(classification_report(y_test, models[i].predict(X_test)))

MODEL:  Random-Forest
              precision    recall  f1-score   support

           0       0.50      0.07      0.12       146
           1       0.64      0.96      0.77       253

    accuracy                           0.63       399
   macro avg       0.57      0.51      0.44       399
weighted avg       0.59      0.63      0.53       399

MODEL:  Decision-Tree
              precision    recall  f1-score   support

           0       0.33      0.32      0.32       146
           1       0.62      0.63      0.62       253

    accuracy                           0.52       399
   macro avg       0.47      0.47      0.47       399
weighted avg       0.51      0.52      0.51       399

MODEL:  GradientBoosting
              precision    recall  f1-score   support

           0       0.38      0.13      0.19       146
           1       0.64      0.88      0.74       253

    accuracy                           0.60       399
   macro avg       0.51      0.50      0.47       399
weigh